In [1]:
import os
import glob
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Embedding, LSTM
from keras.layers import Permute, RepeatVector, Concatenate, Activation
from keras.layers import dot
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import seaborn as sns

Using TensorFlow backend


In [34]:
import csv
train_gt = []
with open('gt_split/train.tsv','r') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    for line in tsv_file:
        if len(line)==2:
            path, exp = line[0], line[1]
            train_gt.append([path, exp])

test_gt = []
with open('gt_split/validation.tsv','r') as file:
    tsv_file = csv.reader(file, delimiter="\t")
    for line in tsv_file:
        if len(line)==2:
            path, exp = line[0], line[1]
            test_gt.append([path, exp])

In [2]:
import pandas as pd
gt_2013 = pd.read_csv('groundtruth_2013.tsv', sep='\t', names=['path', 'gt'])
gt_2014 = pd.read_csv('groundtruth_2014.tsv', sep='\t', names=['path', 'gt'])
gt_2016 = pd.read_csv('groundtruth_2016.tsv', sep='\t', names=['path', 'gt'])

In [3]:
gt_2016

,path,gt
0,UN_108_em_177,y_1 y_2 + y_2 y_1 =0
1,UN_113_em_293,n(xy)=n(x)n(y)
2,UN_117_em_345,\frac {1}{ 360} ( n+3)^2 (n+1)(n+2)(n+4)(n+5)
3,UN_460_em_841,X_7=x_7\sqrt{x_4^4+x_5^4}
4,UN_466_em_993,\frac{1}{2}f-b=8-9=-1
...,...,...
1142,UN_127_em_593,\lim_{r\rightarrow \infty} f(r) = 0
1143,UN_127_em_597,9\mbox{x}9
1144,UN_464_em_928,\frac{T}{L} \log \frac{T}{L}
1145,UN_116_em_335,\frac 1x


In [4]:
combined_df = pd.concat([gt_2013, gt_2014, gt_2016], ignore_index=True)

In [10]:
combined_df

,path,gt
0,NaN,(y-y^{calc})
1,NaN,2sp_{z}
2,rit_4235_3,\frac {5} {6}
3,NaN,\frac {2} {\frac {1} {x} + \frac {1} {y}}
4,NaN,L(w)=\frac{m}{2}||w||^2-V(r)
...,...,...
2799,NaN,\lim_{r\rightarrow \infty} f(r) = 0
2800,UN_127_em_597,9\mbox{x}9
2801,NaN,\frac{T}{L} \log \frac{T}{L}
2802,NaN,\frac 1x


In [5]:
import os
import glob

class_directories = ["test/2013", "test/2014", "test/2016"]

png_file_names = []

for directory in class_directories:
    png_files = glob.glob(os.path.join(directory, "*.png"))
    # Extract just the file names from the full paths
    png_file_names.extend([os.path.splitext(os.path.basename(file))[0] for file in png_files])

print(png_file_names)

# # Now, the png_file_names list contains just the .png file names without the full path
# for png_file_name in png_file_names:
#     print(png_file_name)


['120_em_300', '127_em_508', 'ritm_422_0', 'rit_420_3', 'rit_42100_2', 'rit_42100_3', 'rit_4210_4', 'rit_42110_2', 'rit_42130_0', 'rit_42150_0', 'rit_4215_3', 'rit_42160_2', 'rit_42190_3', 'rit_42190_4', 'rit_4220_1', 'rit_4220_3', 'rit_4225_2', 'rit_4225_3', 'rit_4225_4', 'rit_4230_0', 'rit_4230_2', 'rit_4230_3', 'rit_4235_0', 'rit_4235_2', 'rit_4235_3', 'rit_4235_4', 'rit_4240_0', 'rit_4240_3', 'rit_4240_4', 'rit_4245_2', 'rit_4245_3', 'rit_4250_2', 'rit_4250_3', 'rit_4260_1', 'rit_4260_4', 'rit_4265_0', 'rit_4275_2', 'rit_4280_0', 'rit_4280_2', 'rit_4290_4', 'rit_4295_0', 'rit_4295_1', '18_em_1', '18_em_10', '18_em_13', '18_em_21', '18_em_23', '18_em_24', '20_em_26', '20_em_41', '20_em_42', '20_em_43', '20_em_47', '23_em_53', '23_em_55', '23_em_56', '23_em_57', '23_em_60', '26_em_78', '26_em_93', '26_em_96', '26_em_97', '27_em_100', '27_em_110', '27_em_118', '27_em_119', '27_em_120', '28_em_125', '28_em_133', '28_em_142', '28_em_146', '29_em_159', '31_em_187', '31_em_195', '31_em_19

In [11]:
print(len(png_file_names))

295


In [6]:
import numpy as np

In [7]:
def testing(path, png_file_name):
    if path in png_file_name:
        return path

    return np.nan

In [8]:
combined_df["path"] = combined_df["path"].apply(testing, args=(png_file_names,))

In [9]:
combined_df.isna().sum()

path    2509
gt         0
dtype: int64

In [13]:
combined_df

,path,gt
0,NaN,(y-y^{calc})
1,NaN,2sp_{z}
2,rit_4235_3,\frac {5} {6}
3,NaN,\frac {2} {\frac {1} {x} + \frac {1} {y}}
4,NaN,L(w)=\frac{m}{2}||w||^2-V(r)
...,...,...
2799,NaN,\lim_{r\rightarrow \infty} f(r) = 0
2800,UN_127_em_597,9\mbox{x}9
2801,NaN,\frac{T}{L} \log \frac{T}{L}
2802,NaN,\frac 1x


In [14]:
filtered_df = combined_df.dropna()

In [50]:
filtered_df.head(30)

,path,gt
2,rit_4235_3,\frac {5} {6}
34,rit_4240_0,57753336
51,rit_4210_4,\frac {9} {7}
53,rit_4225_2,8 \sqrt {3}
76,rit_4225_3,\frac {56} {8}
118,rit_4295_0,523 + 487
151,rit_4250_3,\sqrt {18}
161,ritm_422_0,+ 7 \times 9 ^ {5} + 7 \times 9 ^ {4} + 7 \ti...
177,rit_42100_2,\frac {\sqrt {4}} {2}
183,rit_420_3,9 ^ {9 ^ {9 ^ {9 ^ {9 ^ {9}}}}}


In [51]:
filtered_df.to_excel("filtered_v2.xlsx", index=False)

In [ ]:
# Define the Image Encoder
image_input = Input(shape=(image_width, image_height, num_channels))
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
max_pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(64, (3, 3), activation='relu')(max_pool1)
max_pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flatten = Flatten()(max_pool2)
encoder_dense = Dense(256, activation='relu')(flatten)

# Define the LaTeX Decoder
latex_input = Input(shape=(max_latex_length,))
latex_embedding = Embedding(output_vocab_size, embedding_dim)(latex_input)
decoder_lstm = LSTM(256, return_sequences=True)(latex_embedding)

# Attention Mechanism
attn_repeat = RepeatVector(max_latex_length)(encoder_dense)
attn_concat = Concatenate(axis=-1)([decoder_lstm, attn_repeat])
attn_dense1 = Dense(256, activation='tanh')(attn_concat)
attn_score = Dense(1, activation='softmax')(attn_dense1)
attn_dot = dot([attn_score, encoder_dense], axes=(1, 1))
attn_dot = Permute((2, 1))(attn_dot)
context = Concatenate(axis=-1)([decoder_lstm, attn_dot])
decoder_attn = Activation('tanh')(context)

latex_output = Dense(output_vocab_size, activation='softmax')(decoder_attn)

# Connect the Encoder and Decoder
image_to_latex_model = Model(inputs=[image_input, latex_input], outputs=latex_output)

# Compile the Model
image_to_latex_model.compile(optimizer='adam', loss='categorical_crossentropy')

# Summary of the Model
image_to_latex_model.summary()
